# Phase 1: Data Cleaning and Preprocessing

**Project**: House Price Prediction and Analysis Using King County Housing Data

**Team**: Ashwin, Ashwath, Namrata Mane

**Course**: DA 591 - Final Semester Project

---

In this notebook, we will clean and prepare the King County housing dataset for analysis. The steps include:
1. Loading the data
2. Understanding the data structure
3. Checking for missing values
4. Handling duplicates
5. Converting data types
6. Handling outliers
7. Feature engineering
8. Saving the cleaned data

## Step 1: Import Required Libraries

We will use pandas for data manipulation and numpy for numerical operations.

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

## Step 2: Load the Dataset

Loading the King County housing dataset from the CSV file.

In [ ]:
# Load the dataset
df = pd.read_csv('kc_house_data.csv')

# Check how many rows and columns we have
print(f"Dataset loaded successfully!")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

## Step 3: First Look at the Data

Let's see what the data looks like - first few rows and the column names.

In [ ]:
# Display first 5 rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Display last 5 rows to make sure data is complete
print("Last 5 rows of the dataset:")
df.tail()

In [ ]:
# Show all column names
print("Column names in the dataset:")
print(df.columns.tolist())

## Step 4: Understanding Data Types

Let's check the data type of each column and see if any conversions are needed.

In [ ]:
# Check data types of all columns
print("Data types of each column:")
print(df.dtypes)

In [ ]:
# Get more detailed info about the dataset
print("Detailed information about the dataset:")
df.info()

## Step 5: Statistical Summary

Let's look at the basic statistics of numerical columns to understand the data distribution.

In [ ]:
# Statistical summary of numerical columns
print("Statistical summary of the dataset:")
df.describe()

In [ ]:
# Let's look at the price column specifically since it's our target variable
print("Price column statistics:")
print(f"Minimum price: ${df['price'].min():,.2f}")
print(f"Maximum price: ${df['price'].max():,.2f}")
print(f"Average price: ${df['price'].mean():,.2f}")
print(f"Median price: ${df['price'].median():,.2f}")

## Step 6: Check for Missing Values

Missing values can affect our analysis and model performance. Let's check if there are any.

In [ ]:
# Check for missing values in each column
print("Missing values in each column:")
missing_values = df.isnull().sum()
print(missing_values)

In [ ]:
# Calculate percentage of missing values
missing_percentage = (df.isnull().sum() / len(df)) * 100
print("\nPercentage of missing values:")
print(missing_percentage)

In [ ]:
# Total missing values in the entire dataset
total_missing = df.isnull().sum().sum()
print(f"\nTotal missing values in the dataset: {total_missing}")

if total_missing == 0:
    print("Great! No missing values found in the dataset.")
else:
    print("We need to handle these missing values.")

## Step 7: Check for Duplicate Records

Duplicate records can skew our analysis. Let's check if any house is listed more than once.

In [ ]:
# Check for duplicate rows (entire row is same)
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

In [ ]:
# Check for duplicate house IDs
# Same house might be sold multiple times
duplicate_ids = df['id'].duplicated().sum()
print(f"Number of duplicate house IDs: {duplicate_ids}")

if duplicate_ids > 0:
    print(f"\nThis means {duplicate_ids} houses were sold more than once.")
    print("We will keep only the latest sale for each house.")

In [ ]:
# Let's look at some duplicate IDs to understand
if duplicate_ids > 0:
    # Find houses that appear more than once
    duplicate_houses = df[df['id'].duplicated(keep=False)]
    print(f"Total records for houses sold multiple times: {len(duplicate_houses)}")
    
    # Show example of a house sold multiple times
    sample_id = duplicate_houses['id'].iloc[0]
    print(f"\nExample: House ID {sample_id}")
    print(df[df['id'] == sample_id][['id', 'date', 'price']])

## Step 8: Handle Duplicates

For houses sold multiple times, we will keep only the most recent sale since it reflects the current market value better.

In [ ]:
# Store original count
original_count = len(df)
print(f"Original number of records: {original_count}")

# Sort by date in descending order so latest sale comes first
df_sorted = df.sort_values('date', ascending=False)

# Keep only the first occurrence (latest sale) for each house ID
df_cleaned = df_sorted.drop_duplicates(subset='id', keep='first')

# Reset the index
df_cleaned = df_cleaned.reset_index(drop=True)

print(f"Number of records after removing duplicates: {len(df_cleaned)}")
print(f"Records removed: {original_count - len(df_cleaned)}")

In [ ]:
# Verify no more duplicate IDs
remaining_duplicates = df_cleaned['id'].duplicated().sum()
print(f"Remaining duplicate IDs: {remaining_duplicates}")

if remaining_duplicates == 0:
    print("All duplicates have been removed successfully!")

## Step 9: Convert Date Column

The date column is currently stored as a string. Let's convert it to a proper datetime format and extract useful features.

In [ ]:
# Check current format of the date column
print("Sample date values:")
print(df_cleaned['date'].head())
print(f"\nCurrent data type: {df_cleaned['date'].dtype}")

In [ ]:
# Convert date column to datetime
# The format is like '20141013T000000'
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'], format='%Y%m%dT%H%M%S')

print("Date column converted to datetime format:")
print(df_cleaned['date'].head())
print(f"\nNew data type: {df_cleaned['date'].dtype}")

In [ ]:
# Extract useful features from the date
df_cleaned['sale_year'] = df_cleaned['date'].dt.year
df_cleaned['sale_month'] = df_cleaned['date'].dt.month

print("New date features created:")
print(df_cleaned[['date', 'sale_year', 'sale_month']].head())

In [ ]:
# Check the date range in our dataset
print(f"Date range of the data:")
print(f"Earliest sale: {df_cleaned['date'].min()}")
print(f"Latest sale: {df_cleaned['date'].max()}")

## Step 10: Check for Unusual Values

Let's check if there are any unusual or incorrect values in the data that don't make sense.

In [ ]:
# Check bedrooms - can a house have 0 or very high number of bedrooms?
print("Bedroom distribution:")
print(f"Minimum bedrooms: {df_cleaned['bedrooms'].min()}")
print(f"Maximum bedrooms: {df_cleaned['bedrooms'].max()}")
print(f"\nValue counts:")
print(df_cleaned['bedrooms'].value_counts().sort_index())

In [ ]:
# Houses with 0 bedrooms - might be studios or data errors
zero_bedrooms = df_cleaned[df_cleaned['bedrooms'] == 0]
print(f"Number of houses with 0 bedrooms: {len(zero_bedrooms)}")

if len(zero_bedrooms) > 0:
    print("\nThese might be studios or data errors. Let's look at them:")
    print(zero_bedrooms[['id', 'bedrooms', 'bathrooms', 'sqft_living', 'price']].head())

In [ ]:
# Houses with very high bedrooms (more than 10)
high_bedrooms = df_cleaned[df_cleaned['bedrooms'] > 10]
print(f"Number of houses with more than 10 bedrooms: {len(high_bedrooms)}")

if len(high_bedrooms) > 0:
    print("\nThese are unusual. Let's examine them:")
    print(high_bedrooms[['id', 'bedrooms', 'bathrooms', 'sqft_living', 'price']])

In [ ]:
# The house with 33 bedrooms seems like a data entry error
# 33 bedrooms with only 1620 sqft is impossible
# Let's check the sqft per bedroom ratio

if len(high_bedrooms) > 0:
    print("Checking sqft per bedroom for unusual entries:")
    for idx, row in high_bedrooms.iterrows():
        sqft_per_bedroom = row['sqft_living'] / row['bedrooms']
        print(f"House ID {row['id']}: {row['bedrooms']} bedrooms, {row['sqft_living']} sqft")
        print(f"  -> {sqft_per_bedroom:.2f} sqft per bedroom (should be at least 100)")
        
        if sqft_per_bedroom < 80:
            print("  -> This looks like a DATA ERROR!")

In [ ]:
# Fix the obvious data error - house with 33 bedrooms should probably be 3
# Looking at the data: 33 bedrooms, 1.75 bathrooms, 1620 sqft -> clearly an error

error_mask = (df_cleaned['bedrooms'] == 33) & (df_cleaned['sqft_living'] < 2000)
if error_mask.sum() > 0:
    print("Fixing data entry error: 33 bedrooms -> 3 bedrooms")
    df_cleaned.loc[error_mask, 'bedrooms'] = 3
    print("Fixed!")

In [ ]:
# Check for houses with 0 bathrooms
zero_bathrooms = df_cleaned[df_cleaned['bathrooms'] == 0]
print(f"Number of houses with 0 bathrooms: {len(zero_bathrooms)}")

if len(zero_bathrooms) > 0:
    print("\n0 bathrooms is unusual for a house. Let's see:")
    print(zero_bathrooms[['id', 'bedrooms', 'bathrooms', 'sqft_living', 'price']].head())

In [ ]:
# Check sqft_living - should not be 0 or negative
print("Sqft_living check:")
print(f"Minimum: {df_cleaned['sqft_living'].min()}")
print(f"Maximum: {df_cleaned['sqft_living'].max()}")

if df_cleaned['sqft_living'].min() <= 0:
    print("\nWARNING: Found houses with 0 or negative living space!")
else:
    print("\nAll houses have valid living space values.")

## Step 11: Handle Outliers in Price

Extreme outliers can affect our model performance. Let's identify and handle them using the IQR (Interquartile Range) method.

In [ ]:
# Calculate IQR for price
Q1 = df_cleaned['price'].quantile(0.25)
Q3 = df_cleaned['price'].quantile(0.75)
IQR = Q3 - Q1

print("Price distribution:")
print(f"Q1 (25th percentile): ${Q1:,.2f}")
print(f"Q3 (75th percentile): ${Q3:,.2f}")
print(f"IQR: ${IQR:,.2f}")

In [ ]:
# Calculate outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nOutlier boundaries:")
print(f"Lower bound: ${lower_bound:,.2f}")
print(f"Upper bound: ${upper_bound:,.2f}")

In [ ]:
# Count outliers
price_outliers = df_cleaned[(df_cleaned['price'] < lower_bound) | (df_cleaned['price'] > upper_bound)]
print(f"Number of price outliers: {len(price_outliers)}")
print(f"Percentage of data: {(len(price_outliers)/len(df_cleaned))*100:.2f}%")

In [ ]:
# Let's see the distribution of outliers
low_outliers = df_cleaned[df_cleaned['price'] < lower_bound]
high_outliers = df_cleaned[df_cleaned['price'] > upper_bound]

print(f"Houses below lower bound (< ${lower_bound:,.0f}): {len(low_outliers)}")
print(f"Houses above upper bound (> ${upper_bound:,.0f}): {len(high_outliers)}")

In [ ]:
# For this analysis, we will keep most of the data
# Only remove extreme outliers (prices above 3 million or below 50k)
# This is a practical decision since luxury homes are still valid data points

# Let's see how many extreme outliers we have
extreme_high = df_cleaned[df_cleaned['price'] > 3000000]
extreme_low = df_cleaned[df_cleaned['price'] < 50000]

print(f"Extreme outliers:")
print(f"Price > $3,000,000: {len(extreme_high)}")
print(f"Price < $50,000: {len(extreme_low)}")

In [ ]:
# Decision: We will keep all data for now
# Extreme luxury homes are still valid - they represent luxury market
# Very cheap homes might be land or special cases

print("Decision: Keeping all price data for analysis.")
print("Reason: Extreme values represent real market segments (luxury homes, land, etc.)")
print("\nNote: If model performance is poor, we can revisit this decision.")

## Step 12: Feature Engineering

Let's create new features that might be useful for our analysis and modeling.

In [ ]:
# Create 'house_age' feature
# Using the sale year to calculate how old the house was when sold
df_cleaned['house_age'] = df_cleaned['sale_year'] - df_cleaned['yr_built']

print("House age feature created:")
print(f"Youngest house age at sale: {df_cleaned['house_age'].min()} years")
print(f"Oldest house age at sale: {df_cleaned['house_age'].max()} years")
print(f"Average house age at sale: {df_cleaned['house_age'].mean():.1f} years")

In [ ]:
# Create 'renovated' binary feature
# 1 if the house was ever renovated, 0 otherwise
df_cleaned['renovated'] = (df_cleaned['yr_renovated'] > 0).astype(int)

print("Renovated feature created:")
print(df_cleaned['renovated'].value_counts())
print(f"\nPercentage of renovated houses: {df_cleaned['renovated'].mean()*100:.2f}%")

In [ ]:
# Create 'price_per_sqft' feature
df_cleaned['price_per_sqft'] = df_cleaned['price'] / df_cleaned['sqft_living']

print("Price per sqft feature created:")
print(f"Minimum: ${df_cleaned['price_per_sqft'].min():.2f}/sqft")
print(f"Maximum: ${df_cleaned['price_per_sqft'].max():.2f}/sqft")
print(f"Average: ${df_cleaned['price_per_sqft'].mean():.2f}/sqft")

In [ ]:
# Create 'has_basement' binary feature
df_cleaned['has_basement'] = (df_cleaned['sqft_basement'] > 0).astype(int)

print("Has basement feature created:")
print(df_cleaned['has_basement'].value_counts())
print(f"\nPercentage of houses with basement: {df_cleaned['has_basement'].mean()*100:.2f}%")

In [ ]:
# Create 'total_rooms' feature (bedrooms + bathrooms gives a rough idea)
df_cleaned['total_rooms'] = df_cleaned['bedrooms'] + df_cleaned['bathrooms']

print("Total rooms feature created:")
print(f"Minimum total rooms: {df_cleaned['total_rooms'].min()}")
print(f"Maximum total rooms: {df_cleaned['total_rooms'].max()}")
print(f"Average total rooms: {df_cleaned['total_rooms'].mean():.1f}")

## Step 13: Final Data Check

Let's verify our cleaned dataset is ready for analysis.

In [ ]:
# Final shape of the dataset
print("Final dataset summary:")
print(f"Number of rows: {df_cleaned.shape[0]}")
print(f"Number of columns: {df_cleaned.shape[1]}")

In [ ]:
# List all columns in the cleaned dataset
print("All columns in cleaned dataset:")
for i, col in enumerate(df_cleaned.columns, 1):
    print(f"{i}. {col}")

In [ ]:
# Final check for missing values
print("Missing values check:")
missing = df_cleaned.isnull().sum()
if missing.sum() == 0:
    print("No missing values in the cleaned dataset!")
else:
    print(missing[missing > 0])

In [ ]:
# Final check for duplicates
print("Duplicate check:")
if df_cleaned['id'].duplicated().sum() == 0:
    print("No duplicate house IDs!")
else:
    print(f"Warning: {df_cleaned['id'].duplicated().sum()} duplicates found")

In [ ]:
# Display final data types
print("Data types in cleaned dataset:")
print(df_cleaned.dtypes)

In [ ]:
# Preview of the cleaned dataset
print("Preview of cleaned dataset (first 5 rows):")
df_cleaned.head()

## Step 14: Save the Cleaned Dataset

Save the cleaned data to a new CSV file for use in the next phases.

In [ ]:
# Save the cleaned dataset
output_file = 'cleaned_house_data.csv'
df_cleaned.to_csv(output_file, index=False)

print(f"Cleaned dataset saved to: {output_file}")
print(f"Total records: {len(df_cleaned)}")
print(f"Total columns: {len(df_cleaned.columns)}")

In [ ]:
# Verify the saved file
df_verify = pd.read_csv(output_file)
print(f"\nVerification - File loaded successfully!")
print(f"Rows: {len(df_verify)}, Columns: {len(df_verify.columns)}")

## Summary of Data Cleaning

### What we did:
1. **Loaded** the King County housing dataset (21,613 records)
2. **Checked for missing values** - None found
3. **Removed duplicate records** - Kept only the latest sale for houses sold multiple times
4. **Converted date column** - Changed from string to datetime format
5. **Fixed data errors** - Corrected obvious typos (e.g., 33 bedrooms -> 3)
6. **Created new features**:
   - `sale_year` and `sale_month` from date
   - `house_age` (age when sold)
   - `renovated` (binary: 0 or 1)
   - `price_per_sqft`
   - `has_basement` (binary: 0 or 1)
   - `total_rooms`
7. **Saved** the cleaned data to `cleaned_house_data.csv`

### Next Steps:
- Phase 2: Exploratory Data Analysis (EDA)
- Phase 3: Build Linear Regression model for price prediction